In [2]:
from zipline.api import order, record, symbol


def initialize(context):
    pass


def handle_data(context, data):
    order(symbol('AAPL'), 10)
    record(AAPL=data.current(symbol('AAPL'), 'price'))
    
algor_obj = TradingAlgorithm(initialize=initialize, handle_data=handle_data)  
perf_manual = algor_obj.run(data_c)  

NameError: name 'TradingAlgorithm' is not defined

In [4]:
# %%zipline --start 2000-1-1 --end 2012-1-1 -o dma.pickle

from zipline.api import order_target, record, symbol

def initialize(context):
    context.i = 0
    context.asset = symbol('AAPL')


def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < 300:
        return

    # Compute averages
    # data.history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = data.history(context.asset, 'price', bar_count=100, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=300, frequency="1d").mean()

    # Trading logic
    if short_mavg > long_mavg:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(context.asset, 100)
    elif short_mavg < long_mavg:
        order_target(context.asset, 0)

    # Save values for later inspection
    record(AAPL=data.current(context.asset, 'price'),
           short_mavg=short_mavg,
           long_mavg=long_mavg)


def analyze(context, perf):
    fig = plt.figure()
    ax1 = fig.add_subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel('portfolio value in $')

    ax2 = fig.add_subplot(212)
    perf['AAPL'].plot(ax=ax2)
    perf[['short_mavg', 'long_mavg']].plot(ax=ax2)

    perf_trans = perf.ix[[t != [] for t in perf.transactions]]
    buys = perf_trans.ix[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.ix[
        [t[0]['amount'] < 0 for t in perf_trans.transactions]]
    ax2.plot(buys.index, perf.short_mavg.ix[buys.index],
             '^', markersize=10, color='m')
    ax2.plot(sells.index, perf.short_mavg.ix[sells.index],
             'v', markersize=10, color='k')
    ax2.set_ylabel('price in $')
    plt.legend(loc=0)
    plt.show()

In [10]:
#-*- coding: utf-8 -*-  
from datetime import datetime  
from zipline.algorithm import TradingAlgorithm  
from zipline.finance.trading import TradingEnvironment  
from zipline.api import order, record, symbol, history  
from zipline.finance import trading  
from zipline.utils.factory import create_simulation_parameters  
import pandas as pd  
import numpy as np  
  
n = 0  
# Define algorithm  
def initialize(context):  
    context.asset = symbol('AAPL')  
    print ("initialization"  )
    pass  
  
def handle_data(context, data):  
    global n  
    print("handle", n  )
    print (data.history(context.asset, 'price', 1, '1d'))#close price  
    # print history(1, '1d', 'price').mean()  
    n += 1  
    order(symbol('AAPL'), 10)  
    record(AAPL=data.current(symbol('AAPL'), 'price'))  
  
def analyze(context=None, results=None):  
    import matplotlib.pyplot as plt  
    # Plot the portfolio and asset data.  
    ax1 = plt.subplot(211)  
    results.portfolio_value.plot(ax=ax1)  
    ax1.set_ylabel('Portfolio value (USD)')  
    ax2 = plt.subplot(212, sharex=ax1)  
    results.AAPL.plot(ax=ax2)  
    ax2.set_ylabel('AAPL price (USD)')  
  
    # Show the plot.  
    plt.gcf().set_size_inches(18, 8)  
    plt.show()  
  
# 本地化工作开始  
def load_t(trading_day, trading_days, bm_symbol):  
    # dates = pd.date_range('2001-01-01 00:00:00', periods=365, tz="Asia/Shanghai")  
    bm = pd.Series(data=np.random.random_sample(len(trading_days)), index=trading_days)  
    tr = pd.DataFrame(data=np.random.random_sample((len(trading_days), 7)), index=trading_days,  
                      columns=['1month', '3month', '6month', '1year', '2year', '3year', '10year'])  
    return bm, tr  
trading.environment = TradingEnvironment(load=load_t, bm_symbol='^HSI', exchange_tz='Asia/Shanghai')  
  
# 回测参数设置  
sim_params = create_simulation_parameters(year=2014,  
    start=pd.to_datetime("2001-01-01 00:00:00").tz_localize("Asia/Shanghai"),  
    end=pd.to_datetime("2001-09-21 00:00:00").tz_localize("Asia/Shanghai"),  
    data_frequency="daily", emission_rate="daily")  # 原始版本是上面这样的，代码里面是交易日历，然而，如何产生交易日历呢？  
  
  
# setting the algo parameters  
algor_obj = TradingAlgorithm(initialize=initialize, handle_data=handle_data,  
                             sim_params=sim_params, env=trading.environment, analyze=analyze  
                             )  
# data format definition  
parse = lambda x: datetime.date(datetime.strptime(x, '%Y/%m/%d'))  
  
# data generator  
data_s = pd.read_csv('AAPL.csv', parse_dates=['Date'], index_col=0, date_parser=parse)  
print (data_s)  
data_c = pd.Panel({'AAPL': data_s})  
  
perf_manual = algor_obj.run(data_c) 
print(perf_manual)
# Print  
#perf_manual.to_csv('myoutput.csv')  

OSError: File b'AAPL.csv' does not exist

In [ ]:
import pytz  
from datetime import datetime  
  
import zipline  
from zipline.algorithm import TradingAlgorithm  
from zipline.finance.trading import TradingEnvironment  
from zipline.api import order, record, symbol, history  
from zipline.finance import trading  
from zipline.utils.factory import create_simulation_parameters  
from zipline.assets.synthetic import make_simple_equity_info  
#from zipline.utils.calendars import exchange_calendar_sh  
from zipline.utils.calendars import exchange_calendar_lse  
from zipline.utils.calendars import exchange_calendar_nyse  
import pandas as pd  
import numpy as np  
  
n = 0  
# Define algorithm  
def initialize(context):  
    context.asset = symbol('dfqc')  
    print ("initialization"  )
    pass  
  
  
def handle_data(context, data):  
    global n  
    print ("handle", n  )
    print (data.history(context.asset, 'price', 1, '1d'))#close price  
    # print history(1, '1d', 'price').mean()  
    n += 1  
    order(symbol('dfqc'), 10)  
    record(AAPL=data.current(symbol('dfqc'), 'price'))  
  
def analyze(context=None, results=None):  
    import matplotlib.pyplot as plt  
    # Plot the portfolio and asset data.  
    ax1 = plt.subplot(211)  
    results.portfolio_value.plot(ax=ax1)  
    ax1.set_ylabel('Portfolio value (USD)')  
    ax2 = plt.subplot(212, sharex=ax1)  
    results.AAPL.plot(ax=ax2)  
    ax2.set_ylabel('dfqc price (USD)')  
  
    # Show the plot.  
    plt.gcf().set_size_inches(18, 8)  
    plt.show()  
  
# This is needed to handle the correct calendar. Assume that market data has the right index for trade able days.  
# Passing in env_trading_calendar=trading calendar_lse doesn't appear to work, as it doesn't implement open_and_closes  
  
def load_t(trading_day, trading_days, bm_symbol):  
    # dates = pd.date_range('2001-01-01 00:00:00', periods=365, tz="Asia/Shanghai")  
    bm = pd.Series(data=np.random.random_sample(len(trading_days)), index=trading_days)  
    tr = pd.DataFrame(data=np.random.random_sample((len(trading_days), 7)), index=trading_days,  
                      columns=['1month', '3month', '6month', '1year', '2year', '3year', '10year'])  
    return bm, tr  
#cal = exchange_calendar_sh.SHExchangeCalendar()  
trading.environment = TradingEnvironment(load=load_t, bm_symbol='^HSI', exchange_tz='Asia/Shanghai', trading_calendar=cal)  
  
  
  
# 回测参数设置  
sim_params = create_simulation_parameters(year=2014,  
    start=pd.to_datetime("2014-01-01 00:00:00").tz_localize("Asia/Shanghai"),  
    end=pd.to_datetime("2014-03-07 00:00:00").tz_localize("Asia/Shanghai"),  
    data_frequency="daily", emission_rate="daily")  # 原始版本是上面这样的，代码里面是交易日历，然而，如何产生交易日历呢？  
  
  
# setting the algo parameters  
algor_obj = TradingAlgorithm(initialize=initialize, handle_data=handle_data,  
                             sim_params=sim_params, env=trading.environment, analyze=analyze,  
                             trading_calendar=cal)  
  
# data format definition  
parse = lambda x: datetime.date(datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))  
  
# parse = lambda x: datetime.date(pytz.utc.localize(datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')))  
# data generator  
data_s = pd.read_csv('dfqc.csv', parse_dates=['date'], index_col=0, date_parser=parse)  
print (data_s ) 
data_c = pd.Panel({'dfqc': data_s})  
  
  
perf_manual = algor_obj.run(data_c)  
# Print  
#perf_manual.to_csv('myoutput.csv')  
print(perf_manual)

NameError: name 'exchange_calendar_sh' is not defined

In [6]:
from datetime import time  
from itertools import chain  
  
from pandas.tseries.holiday import (  
    GoodFriday,  
    USPresidentsDay,  
    USLaborDay,  
    USThanksgivingDay  
)  
from pandas.tslib import Timestamp  
from pytz import timezone  
from pandas.tseries.holiday import (  
    Holiday,  
    DateOffset,  
    MO,  
    weekend_to_monday,  
    GoodFriday,  
    EasterMonday,  
)  
  
from zipline.utils.calendars import TradingCalendar  
from zipline.utils.calendars.trading_calendar import HolidayCalendar  
from zipline.utils.calendars.us_holidays import (  
    USNewYearsDay,  
    Christmas,  
    USMartinLutherKingJrAfter1998,  
    USMemorialDay,  
    USIndependenceDay,  
    USNationalDaysofMourning)  
  
CHINANewYearsDay = Holiday(  
    "New Year's Day",  
    month=1,  
    day=31,  
)  
class SHExchangeCalendar(TradingCalendar):  
    @property  
    def name(self):  
        return "SH"  
 
    @property  
    def tz(self):  
        return timezone("Asia/Shanghai")  
 
    @property  
    def open_time(self):  
        return time(9, 30)  
 
    @property  
    def close_time(self):  
        return time(15)  
 
    @property  
    def regular_holidays(self):  
        return HolidayCalendar([  
            CHINANewYearsDay  
        ]) 